In [1]:
# extract the fingerprints from the h5 SpecUFEx file,
# run k-means on them and merge with a catalog
import sys

import numpy as np
import h5py
from sklearn.metrics import silhouette_samples
from sklearn.cluster import KMeans
import pandas as pd

from importlib import reload
from specufex_processing.clustering import functions_clustering as funclust

import yaml
import argparse
import os


In [13]:
import importlib as oe

oe.reload(funclust)

<module 'specufex_processing.clustering.functions_clustering' from '/home/tmittal/Dropbox/Projects/gsm/specufex_processing/specufex_processing/clustering/functions_clustering.py'>

In [15]:
with open("config_test1.yml", 'r') as yamlfile:
    config = yaml.load(yamlfile, Loader=yaml.FullLoader)


# pull out config values for conciseness
path_config = config["paths"]
projectPath = path_config["projectPath"]
key = path_config["key"]

data_config = config['dataParams']
station = data_config["station"]
channel = data_config["channel"]
channel_ID = data_config["channel_ID"]
sampling_rate = data_config["sampling_rate"]


clustering_params = config['clusteringParams']
# build path strings
dataH5_name = f'data_{key}.h5'
dataH5_path = os.path.join(projectPath,'H5files/', dataH5_name)

SpecUFEx_H5_name = 'SpecUFEx_' + path_config["h5name"]
SpecUFEx_H5_path = os.path.join(projectPath, 'H5files/', SpecUFEx_H5_name)
pathWf_cat  = os.path.join(projectPath, 'wf_cat_out.csv')

sgram_catPath = os.path.join(projectPath,f'sgram_cat_out_{key}.csv')
cat0 = pd.read_csv(sgram_catPath)

path_cluster_base = os.path.join(config['paths']['projectPath'],"clustering_Catalog")
os.makedirs(path_cluster_base,exist_ok=True)

#### After linearizing FPs into data array X, you can then do Kmeans on X
## Optimal cluster is chosen by mean Silh scores, but euclidean distances are saved also
X,evID_hdf5,orig_df = funclust.linearizeFP(SpecUFEx_H5_path,cat0)

# ====================================================================
# Do the PCA (in case of clustering on PCA but also for visualization)
# ====================================================================

# # Clustering parameters
if clustering_params['use_PCA']:
    if clustering_params['use_PCA_method'] == 'num':
        PCA_df, Y_pca = funclust.PCAonFP(X,evID_hdf5,cat0,
                                            numPCA=clustering_params['numPCA'],
                                            normalization=True)
        X_use = Y_pca
        df_use = PCA_df
    elif clustering_params['use_PCA_method'] == 'var':
        PCA_df, Y_pca = funclust.PVEofPCA(X,evID_hdf5,cat0,
                                            cum_pve_thresh=clustering_params['cum_pve_thresh'],
                                            normalization=True)
        X_use = Y_pca
        df_use = PCA_df
    else :
        print(f"{clustering_params['use_PCA_method']} needs to num or var")
else :
        X_use = X
        df_use = orig_df

if clustering_params['runSilhouette'] == 'True':
    Kmax = clustering_params['Kmax']
    range_n_clusters = range(2,Kmax+1)

    # NOTE that i modified this so that you pass in X, be it fingerprints or PCA-- do that outside the function.
    df_use,catall_euc,catall_ss,Kopt, maxSilScore, avgSils,centers,sse = funclust.calcSilhScore(X_use,df_use,range_n_clusters,topF=clustering_params['topF'])
    catall_euc.to_csv(path_cluster_base+f"/Clustering_Kmeans_NC{Kopt}_Catalog_Sorted_By_Euc_Top{clustering_params['topF']}.csv")
    catall_ss.to_csv(path_cluster_base+f"/Clustering_Kmeans_NC{Kopt}_Catalog_Sorted_By_SS_Top{clustering_params['topF']}.csv")

else :
    ## use the passed values of K_vals
    sse = []
    centers = []
    for K_save in clustering_params['K_vals']:
        print(f"Rerunning kmeans on {K_save} clusters, to save to catalog")
        kmeans = KMeans(n_clusters=K_save,
                           max_iter = 500,
                           init='k-means++', #how to choose init. centroid
                           n_init=10, #number of Kmeans runs
                           random_state=0) #set rand state
        #get cluster labels
        cluster_labels_0 = kmeans.fit_predict(X_use)
        #increment labels by one to match John's old kmeans code
        cluster_labels = [int(ccl)+1 for ccl in cluster_labels_0]
        #get euclid dist to centroid for each point
        sqr_dist = kmeans.transform(X_use)**2 #transform X to cluster-distance space.
        sum_sqr_dist = sqr_dist.sum(axis=1)
        euc_dist = np.sqrt(sum_sqr_dist)
        #save centroids
        centers.append(kmeans.cluster_centers_ )
        #kmeans loss function
        sse.append(kmeans.inertia_)
        sample_silhouette_values = silhouette_samples(X, cluster_labels)
        df_use[f'Cluster_NC{K_save}'] = cluster_labels
        df_use[f'SS_NC{K_save}'] = sample_silhouette_values
        df_use[f'euc_dist_NC{K_save}'] = euc_dist
        catall_euc,catall_ss = funclust.getTopF_labels(K_save,df_use,topF=clustering_params['topF'])
        catall_euc.to_csv(path_cluster_base+f"/Clustering_Kmeans_NC{Kopt}_Catalog_Sorted_By_Euc_Top{clustering_params['topF']}.csv")
        catall_ss.to_csv(path_cluster_base+f"/Clustering_Kmeans_NC{Kopt}_Catalog_Sorted_By_SS_Top{clustering_params['topF']}.csv")

df_use.to_csv(path_cluster_base+'/Clustering_Kmeans_Catalog.csv')

original_stdout = sys.stdout # Save a reference to the original standard output
with open(path_cluster_base+'/Clustering_Kmeans_Catalog_Info.txt', 'w') as f:
    sys.stdout = f # Change the standard output to the file we created.
    print('Parameters of the clustering Algorithm')
    print(clustering_params)
    print(f'Cluster Number {range_n_clusters}')
    print('kmeans loss function : ')
    print(sse)
    print('kmeans centroids : ')
    print(centers)
    sys.stdout = original_stdout # Reset the standard output to its original value


Using 2 PCA components with 0.8656409228618773 variance explained
Rerunning kmeans on 2 clusters, to save to catalog
Rerunning kmeans on 3 clusters, to save to catalog
Rerunning kmeans on 4 clusters, to save to catalog
Rerunning kmeans on 5 clusters, to save to catalog


In [16]:
df_use

,Unnamed: 0,ev_ID,timestamp,filename,PC1,PC2,Cluster_NC2,SS_NC2,euc_dist_NC2,Cluster_NC3,SS_NC3,euc_dist_NC3,Cluster_NC4,SS_NC4,euc_dist_NC4,Cluster_NC5,SS_NC5,euc_dist_NC5
0,0,10633,1984-03-15 16:16:12.020,BAV.NC.EHZ..D.1984.075.161611.10633,-0.077018,0.001970,1,0.876796,0.248904,1,0.841260,0.454347,1,1.000000,0.474804,1,1.0,0.515658
1,1,13436,1984-04-02 20:24:29.880,BAV.NC.EHZ..D.1984.093.202430.13436,-0.077018,0.001970,1,0.876796,0.248904,1,0.841260,0.454347,1,1.000000,0.474804,1,1.0,0.515658
2,2,14421,1984-04-08 11:37:44.930,BAV.NC.EHZ..D.1984.099.113744.14421,-0.077018,0.001970,1,0.876796,0.248904,1,0.841260,0.454347,1,1.000000,0.474804,1,1.0,0.515658
3,3,14423,1984-04-08 11:53:20.200,BAV.NC.EHZ..D.1984.099.115320.14423,0.070270,-0.115838,2,0.070314,0.228689,2,0.685761,0.374792,2,0.669821,0.459752,5,0.0,0.460705
4,4,14460,1984-04-08 19:42:53.340,BAV.NC.EHZ..D.1984.099.194253.14460,0.323572,0.077361,2,0.205159,0.446932,3,0.000000,0.500784,3,0.000000,0.624623,3,0.0,0.694697
5,5,16940,1984-04-29 11:08:56.560,BAV.NC.EHZ..D.1984.120.110856.16940,-0.077018,0.001970,1,0.876796,0.248904,1,0.841260,0.454347,1,1.000000,0.474804,1,1.0,0.515658
6,6,18815,1984-05-18 07:41:32.050,BAV.NC.EHZ..D.1984.139.074132.18815,-0.077018,0.001970,1,0.876796,0.248904,1,0.841260,0.454347,1,1.000000,0.474804,1,1.0,0.515658
7,7,19578,1984-05-28 12:39:26.640,BAV.NC.EHZ..D.1984.149.123926.19578,0.103612,-0.108229,2,0.203678,0.233646,2,0.710654,0.360998,2,0.699618,0.454208,2,0.0,0.455173
8,8,23431,1984-07-20 08:47:43.720,BAV.NC.EHZ..D.1984.202.084743.23431,-0.077018,0.001970,1,0.876796,0.248904,1,0.841260,0.454347,1,1.000000,0.474804,1,1.0,0.515658
9,9,24966,1984-08-05 07:19:05.910,BAV.NC.EHZ..D.1984.218.071905.24966,-0.035348,0.134889,1,0.361708,0.297494,1,0.272355,0.471502,4,0.000000,0.476930,4,0.0,0.551337
